In [ ]:
#import dependencies
import pandas as pd
from bs4 import BeautifulSoup as bs
import requests
import pymongo 
from sqlalchemy import create_engine 
import numpy as np
from matplotlib import pyplot as plt
# import scipy.stats as st
# conn = 'mongodb://localhost:27017'
# client = pymongo.MongoClient(conn)

In [ ]:
filepath = 'resources/alcoholSubstanceAbuse.csv'

In [ ]:
df = pd.read_csv(filepath)
df = df.rename(columns={'Location':'Country','Dim1':'Sex','First Tooltip':'Total alcohol per capita'})
df = df.drop(columns=['Indicator'])
# df.groupby(['Country', 'Sex'])
df.head(20)

In [ ]:
# idx = pd.IndexSlice 
# df = df.loc[idx[:,[2018],['Both sexes'],:],:]

df = df[(df['Period']==2018) & (df['Sex']== 'Both sexes')]
df

In [ ]:
url = 'https://en.wikipedia.org/wiki/List_of_countries_by_GDP_(nominal)_per_capita'
raw = pd.read_html(url)

In [ ]:
df_percapita = raw[0]
df_percapita
df_percapita = df_percapita.drop([0,1,2])
df_percapita = df_percapita.rename(columns={0:'Rank',1:'Country', 2:'GDP Per Capita'})
df_percapita.reset_index(drop=True, inplace=True)
# df_percapita.drop(indexNames , inplace=True)

# Let’s delete all rows for which column ‘Age’ has value 30 i.e.
# Get names of indexes for which column Age has value 30
indexNames = df_percapita[df_percapita['GDP Per Capita'] == 'US$'].index
# indexNames = df_percapita[df_percapita['GDP Per Capita'] == 'nan'].index == this line does not work

# Delete these row indexes from dataFrame
df_percapita.drop(indexNames , inplace=True)

df_percapita.dropna(subset=['GDP Per Capita'], inplace=True)
# df_percapita = df_percapita.drop(df_percapita.loc[df_percapita['GDP Per Capita']=='US$'], inplace=True)
# df_percapita = df_percapita.groupby('Country')['GDP Per Capita'].mean()
# df_percapita = df_percapita.mean('GDP Per Capita')
# groupby country and avg per capita
# df_percapita['GDP Per Capita'].dtypes
# use df["col"]. astype(str). astype(int)
df_percapita["GDP Per Capita"] = df_percapita["GDP Per Capita"].astype(str).astype(int)
df_percapita = df_percapita.groupby('Country')['GDP Per Capita'].mean()
df_percapita

In [ ]:
# df_percapita['gdp_per_capita'].unique() check for values

In [ ]:
# merge dfs based on 'country'
new_df = pd.merge(df, df_percapita, on='Country')
# new_df = df_percapita.groupby('Country')['GDP Per Capita'].mean()
new_df = new_df.rename(columns={'Country':'country', 'Period':'period', 'Sex':'sex', 'Total alcohol per capita':'total_alcohol_per_capita','GDP Per Capita':'gdp_per_capita'})
new_df

In [ ]:
rds_connection_string = "postgres:postgres@localhost:5432/etl_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [ ]:
engine.table_names()

In [ ]:
# engine = create_engine(f'postgresql://{rds_connection_string}')
new_df.to_sql(name="gdp_alcohol", con=engine, if_exists='append', index=False)

In [ ]:
# x=new_df['gdp_per_capita']
# y=new_df['total_alcohol_per_capita']
# correlation = st.pearsonr(x,y)
# correlation
# # plt.show()

In [ ]:
# seaborn heatmap // group data //  gdp < >  